# LangFuse: Monitoring und Tracing von LLM-Anwendungen

In diesem Notebook lernen Sie, wie Sie LangFuse für Monitoring, Tracing und Evaluierung Ihrer LLM-Anwendungen einsetzen können. LangFuse ist ein Open-Source-Tool, das Ihnen hilft, die Leistung und Qualität Ihrer KI-Anwendungen zu überwachen und zu verbessern.

## Überblick

- Einführung in LangFuse
- Installation und Konfiguration
- Einfaches Tracing einer LLM-Anfrage
- Integration mit LangChain
- Traces mit Spans und Generations
- Metriken und Evaluierungsmöglichkeiten
- Praktische Anwendungsfälle

## Voraussetzungen

Um diesem Notebook zu folgen, benötigen Sie:

- LangFuse-Konto (entweder LangFuse Cloud oder selbst gehostete Instanz)
- LangFuse API-Schlüssel
- OpenAI API-Schlüssel

Wenn Sie noch kein LangFuse-Konto haben, können Sie sich [hier](https://cloud.langfuse.com) registrieren.

## 1. Installation der erforderlichen Pakete

Zunächst müssen wir die erforderlichen Pakete installieren. Wir benötigen die LangFuse-Bibliothek und die LangChain-Integration mit LangFuse.

In [5]:
# Installation der benötigten Bibliotheken
!pip install langfuse openai langchain -q


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## 2. Umgebungsvariablen konfigurieren

Wir müssen unsere API-Schlüssel konfigurieren. In einer Produktionsumgebung würden Sie diese als Umgebungsvariablen setzen. Für dieses Notebook werden wir sie direkt im Code definieren.

In [7]:
import os
from dotenv import load_dotenv

# Laden der Umgebungsvariablen aus einer .env-Datei (falls vorhanden)
load_dotenv()

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

# LangFuse-Konfiguration
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY", "")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY", "")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST", "https://cloud.langfuse.com")

# OpenAI-Konfiguration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

# Überprüfen, ob die API-Schlüssel gesetzt sind
if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    print("Bitte setzen Sie die LangFuse API-Schlüssel")
    
if not OPENAI_API_KEY:
    print("Bitte setzen Sie den OpenAI API-Schlüssel")

Bitte setzen Sie die LangFuse API-Schlüssel


## 3. Einfaches Tracing mit LangFuse

Beginnen wir mit einem einfachen Beispiel, um zu verstehen, wie LangFuse funktioniert. Wir werden einen Trace erstellen und eine LLM-Anfrage verfolgen.

In [8]:
from langfuse import Langfuse

# Initialisieren des LangFuse-Clients
langfuse = Langfuse(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

# Einen Trace erstellen
trace = langfuse.trace(
    name="einfaches_beispiel",
    metadata={"beispiel_typ": "einfach", "benutzer_id": "demo-user-123"}
)

print(f"Trace erstellt mit ID: {trace.id}")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Trace erstellt mit ID: f7707b72-55f6-4b80-9abe-ef29ece9036e


Nun fügen wir eine Generation zu diesem Trace hinzu. Eine Generation ist eine Aufzeichnung einer LLM-Anfrage und der entsprechenden Antwort.

In [ ]:
# Eine Generation zum Trace hinzufügen
generation = trace.generation(
    name="erste_anfrage",
    model="gpt-3.5-turbo",
    prompt="Erkläre in einem kurzen Absatz, was künstliche Intelligenz ist.",
    completion="Künstliche Intelligenz (KI) bezeichnet Systeme, die menschenähnliche Intelligenz simulieren können, indem sie Daten analysieren, Muster erkennen und selbstständig Entscheidungen treffen. Diese Technologie umfasst Bereiche wie maschinelles Lernen, natürliche Sprachverarbeitung und Computer Vision und findet Anwendung in zahlreichen Bereichen von virtuellen Assistenten bis hin zu autonomen Fahrzeugen.",
    metadata={"zweck": "einfache_erklaerung", "zielgruppe": "anfaenger"}
)

print(f"Generation hinzugefügt mit ID: {generation.id}")

Wir können auch einen Span hinzufügen, um eine längere Operation zu verfolgen. Ein Span ist eine zeitlich begrenzte Operation, die Teil eines Traces ist.

In [ ]:
import time

# Einen Span zum Trace hinzufügen
span = trace.span(name="datenverarbeitung")

# Eine Operation simulieren
print("Verarbeite Daten...")
time.sleep(1.5)  # Simulation einer zeitaufwändigen Operation

# Den Span beenden
span.end()

print(f"Span beendet mit ID: {span.id}")

Schließlich können wir den Trace beenden und Feedback hinzufügen.

In [ ]:
# Feedback zum Trace hinzufügen
feedback = langfuse.feedback(
    trace_id=trace.id,
    name="klarheit",
    value=0.9,  # Wert zwischen 0 und 1
    comment="Die Erklärung war sehr klar und leicht verständlich."
)

print(f"Feedback hinzugefügt mit ID: {feedback.id}")

# Trace beenden
trace.update(status="success")

print(f"Trace erfolgreich aktualisiert: {trace.id}")

## 4. Reale LLM-Interaktion mit LangFuse Tracing

Jetzt werden wir eine echte Interaktion mit OpenAI durchführen und diese mit LangFuse verfolgen.

In [ ]:
from openai import OpenAI
import time

# OpenAI-Client initialisieren
client = OpenAI(api_key=OPENAI_API_KEY)

# Neuen Trace erstellen
trace_real = langfuse.trace(name="echte_llm_anfrage")

# Zeit messen
start_time = time.time()

# Einen Span für die gesamte Operation erstellen
with trace_real.span(name="vollstaendige_operation") as operation_span:
    
    # Prompt vorbereiten
    with operation_span.span(name="prompt_vorbereitung") as prompt_span:
        prompt = """Du bist ein hilfreicher Assistent für Python-Programmierung. 
        Bitte erstelle eine einfache Funktion, die überprüft, 
        ob eine Zahl eine Primzahl ist."""
        prompt_span.end()
    
    # LLM-Anfrage durchführen und als Generation aufzeichnen
    with operation_span.span(name="llm_anfrage") as llm_span:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Du bist ein hilfreicher Python-Programmierassistent."},
                {"role": "user", "content": prompt}
            ]
        )
        
        # Antwort extrahieren
        completion = response.choices[0].message.content
        llm_span.end()
    
    # Die Generation zum Trace hinzufügen
    generation = trace_real.generation(
        name="primzahl_funktion",
        model="gpt-3.5-turbo",
        prompt=prompt,
        completion=completion,
        usage={
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        },
        metadata={
            "zweck": "code_generation",
            "programmiersprache": "python",
            "aufgabe": "primzahl_prüfung"
        }
    )

# Gesamtzeit berechnen    
end_time = time.time()
duration = end_time - start_time

# Trace aktualisieren
trace_real.update(
    status="success",
    metadata={
        "dauer_sekunden": duration,
        "token_gesamt": response.usage.total_tokens
    }
)

print("LLM-Antwort:")
print(completion)
print(f"\nTrace ID: {trace_real.id}")
print(f"Dauer: {duration:.2f} Sekunden")
print(f"Gesamt-Tokens: {response.usage.total_tokens}")

## 5. Integration mit LangChain

LangFuse lässt sich nahtlos in LangChain integrieren, um Chains, Agents und andere LangChain-Komponenten zu verfolgen.

In [ ]:
from langchain.callbacks import LangfuseCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# LangFuse Callback-Handler erstellen
langfuse_handler = LangfuseCallbackHandler(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

# LLM mit Callback-Handler initialisieren
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    callbacks=[langfuse_handler]
)

# Prompt-Template erstellen
template = ChatPromptTemplate.from_messages([
    SystemMessage(content="Du bist ein Experte für {thema}. Beantworte die folgende Frage."),
    HumanMessage(content="{frage}")
])

# Chain erstellen
chain = LLMChain(llm=llm, prompt=template)

# Chain ausführen
response = chain.invoke({
    "thema": "deutsche Geschichte",
    "frage": "Was waren die wichtigsten Ereignisse beim Fall der Berliner Mauer?"
})

print("LangChain-Antwort:")
print(response["text"])

## 6. Fortgeschrittenes Tracing einer mehrschichtigen Anwendung

Jetzt werden wir eine komplexere Anwendung mit mehreren Schritten erstellen und mit LangFuse verfolgen. Dies simuliert einen typischen RAG-Workflow (Retrieval Augmented Generation).

In [ ]:
from langfuse.client import StatelessTracer

# Stateless Tracer für komplexere Anwendungen
tracer = StatelessTracer(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

# Einen neuen Trace für die RAG-Anwendung erstellen
trace_rag = tracer.trace(name="rag_anwendung", tags=["demo", "rag", "workshop"])

# Benutzerfrage
query = "Was sind die Vorteile von Vektordatenbanken für RAG-Anwendungen?"

# Simulierter RAG-Workflow
with trace_rag.span(name="rag_workflow") as rag_span:
    
    # 1. Schritt: Query-Verständnis
    with rag_span.span(name="query_verstaendnis") as query_span:
        # Simulieren der Query-Analyse
        print("Analysiere Anfrage...")
        time.sleep(0.5)
        
        # Generierung für die Query-Analyse
        query_analysis = trace_rag.generation(
            name="query_analyse",
            model="gpt-3.5-turbo",
            prompt=f"Analysiere die folgende Frage und extrahiere die Hauptthemen: {query}",
            completion="Hauptthemen: Vektordatenbanken, RAG-Anwendungen, Vorteile/Nutzen",
            metadata={"schritt": "1_analyse"}
        )
        query_span.end()
    
    # 2. Schritt: Dokumenten-Retrieval
    with rag_span.span(name="dokument_retrieval") as retrieval_span:
        print("Hole relevante Dokumente aus der Vektordatenbank...")
        time.sleep(1.0)
        
        # Simulieren von gefundenen Dokumenten
        retrieved_docs = [
            "Vektordatenbanken speichern Embeddings für schnelle Ähnlichkeitssuche...",
            "RAG-Anwendungen kombinieren Retrieval mit generativen Modellen...",
            "Vorteile von Vektordatenbanken sind Geschwindigkeit und semantische Suche..."
        ]
        
        # Eigenschaften des Retrievals aufzeichnen
        retrieval_span.update(
            metadata={
                "anzahl_dokumente": len(retrieved_docs),
                "retrieval_methode": "vector_similarity",
                "top_k": 3
            }
        )
        retrieval_span.end()
    
    # 3. Schritt: Prompt-Engineering für RAG
    with rag_span.span(name="prompt_engineering") as prompt_span:
        print("Erstelle RAG-Prompt...")
        
        # Erstellen des RAG-Prompts
        context = "\n\n".join(retrieved_docs)
        rag_prompt = f"""Basierend auf folgenden Informationen, beantworte die Frage.
        
        KONTEXT:
        {context}
        
        FRAGE:
        {query}
        """
        
        prompt_span.end()
    
    # 4. Schritt: Generierung der Antwort
    with rag_span.span(name="antwort_generierung") as generation_span:
        print("Generiere Antwort mit LLM...")
        time.sleep(1.2)
        
        # Simulierte Antwortgenerierung
        answer = """Vektordatenbanken bieten mehrere entscheidende Vorteile für RAG-Anwendungen:
        
        1. Geschwindigkeit: Sie ermöglichen schnelle Ähnlichkeitssuchen in großen Datenmengen
        2. Semantische Suche: Sie finden Dokumente basierend auf Bedeutung, nicht nur auf exakten Wortübereinstimmungen
        3. Skalierbarkeit: Sie können effizient mit wachsenden Datenmengen umgehen
        4. Flexibilität: Sie unterstützen verschiedene Embedding-Modelle und Ähnlichkeitsmaße
        5. Filterungsmöglichkeiten: Sie ermöglichen die Kombination von Vektorsuche mit Metadatenfilterung
        
        Diese Eigenschaften machen Vektordatenbanken zum idealen Speichermechanismus für RAG-Anwendungen, da sie die relevantesten Informationen für die Eingabeaufforderung des generativen Modells liefern können."""
        
        # Aufzeichnen der Generierung
        final_generation = trace_rag.generation(
            name="finale_antwort",
            model="gpt-4",
            prompt=rag_prompt,
            completion=answer,
            metadata={
                "schritt": "4_generierung",
                "kontext_laenge": len(context),
                "verwendete_dokumente": len(retrieved_docs)
            }
        )
        generation_span.end()

# Trace abschließen
trace_rag.update(status="success")

print("\nFertige Antwort:")
print(answer)
print(f"\nRAG-Trace ID: {trace_rag.id}")

## 7. Metriken und Evaluierung mit LangFuse

LangFuse ermöglicht es nicht nur, Anfragen zu verfolgen, sondern auch, die Qualität und Leistung zu bewerten.

In [ ]:
# 1. Feedback-Beispiel: Benutzerfeedback hinzufügen
user_feedback = langfuse.feedback(
    trace_id=trace_rag.id,
    name="nuetzlichkeit",
    value=0.85,  # 0-1 Skala
    comment="Die Antwort war informativ und gut strukturiert."
)

print(f"Benutzerfeedback hinzugefügt: {user_feedback.id}")

# 2. Automatische Evaluierung mit einem LLM
eval_prompt = f"""
Bewerte die folgende Antwort auf die Frage: "{query}"

ANTWORT:
{answer}

Bewerte die Antwort auf einer Skala von 0 bis 10 in folgenden Kategorien:
1. Relevanz zur Frage
2. Korrektheit der Informationen
3. Vollständigkeit
4. Klarheit und Verständlichkeit
"""

eval_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Du bist ein kritischer Evaluator für KI-generierte Antworten."},
        {"role": "user", "content": eval_prompt}
    ]
)

eval_result = eval_response.choices[0].message.content

# Feedback basierend auf der automatischen Evaluierung hinzufügen
auto_feedback = langfuse.feedback(
    trace_id=trace_rag.id,
    name="llm_evaluierung",
    comment=eval_result,
    metadata={"evaluierung_methode": "llm_as_judge", "evaluator_model": "gpt-3.5-turbo"}
)

print("\nAutomatische Evaluierung:")
print(eval_result)
print(f"Evaluierungs-Feedback-ID: {auto_feedback.id}")

## 8. Analyse und Visualisierung in LangFuse UI

Die bisher erstellten Traces können in der LangFuse-Benutzeroberfläche angesehen werden.

- Besuchen Sie [LangFuse Cloud](https://cloud.langfuse.com) (oder Ihre selbst gehostete Instanz)
- Melden Sie sich mit Ihren Zugangsdaten an
- Navigieren Sie zum Bereich "Traces"
- Suchen Sie nach den erstellten Trace-IDs oder dem Namen "rag_anwendung"

In der UI können Sie:

1. Den vollständigen Ablauf eines Traces mit allen Spans und Generations sehen
2. Die Dauer jedes Schritts analysieren
3. Prompts und Completions im Detail betrachten
4. Metriken wie Token-Nutzung und Kosten einsehen
5. Feedback und Evaluierungen überprüfen

![LangFuse UI Beispiel](https://docs.langfuse.com/img/ui/dataset-detail.jpg)

*Bild: Beispiel einer LangFuse-Trace-Visualisierung (Quelle: LangFuse-Dokumentation)*

## 9. Analysen und Dashboards

LangFuse bietet verschiedene Analysemöglichkeiten, um die Leistung Ihrer KI-Anwendungen zu überwachen und zu verbessern.

### Wichtige Metriken in LangFuse

1. **Latenz**: Wie schnell antwortet Ihre Anwendung?
2. **Kosten**: Wie viel kosten Ihre LLM-Anfragen?
3. **Token-Nutzung**: Wie viele Tokens verbrauchen Ihre Prompts und Antworten?
4. **Qualität**: Wie gut sind die generierten Antworten basierend auf Feedback?
5. **Trace-Vollständigkeit**: Wie durchgängig ist Ihr Tracing?

### Einsatz im Entwicklungszyklus

- **Entwicklung**: Debugging und Optimierung von Prompts
- **Qualitätssicherung**: Identifizierung von Schwachstellen
- **Produktion**: Überwachung von Leistung und Kosten
- **Verbesserung**: Datengestützte Entscheidungen für Verbesserungen

### Praktische Tipps zur Nutzung von LangFuse-Dashboards

1. Erstellen Sie benutzerdefinierte Dashboards für verschiedene Anwendungsfälle
2. Setzen Sie Alerts für ungewöhnliche Metriken wie hohe Tokenverbrauch oder Latenz
3. Vergleichen Sie verschiedene Modelle und Prompt-Strategien
4. Nutzen Sie Datasets für systematische Evaluierungen
5. Exportieren Sie Daten für weiterführende Analysen


## 10. Beispiel: A/B-Testing mit LangFuse

Ein wichtiger Anwendungsfall für LangFuse ist A/B-Testing verschiedener Modelle oder Prompt-Strategien.

In [ ]:
import random

# A/B-Test für verschiedene Prompt-Strukturen
test_query = "Erkläre den Unterschied zwischen Supervised und Unsupervised Learning."

# Zwei verschiedene Prompt-Strategien
prompt_a = f"""Als Experte für maschinelles Lernen, erkläre den Unterschied zwischen 
Supervised und Unsupervised Learning. Gib Beispiele für beide Ansätze."""

prompt_b = f"""Definiere kurz und präzise:
1. Supervised Learning
2. Unsupervised Learning
3. Hauptunterschiede
4. Typische Anwendungsfälle für jeden Ansatz"""

# Test-IDs zur Identifizierung der Varianten
test_id = f"prompt_test_{int(time.time())}"

# Funktion zum Durchführen eines Tests
def run_test_variant(variant, prompt):
    # Trace mit Test-Metadaten erstellen
    trace = langfuse.trace(
        name="ab_test_ml_erklaerung",
        tags=["ab_test", f"variant_{variant}"],
        metadata={
            "test_id": test_id,
            "variant": variant,
            "query": test_query
        }
    )
    
    # LLM-Anfrage
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Du bist ein Experte für maschinelles Lernen."},
            {"role": "user", "content": prompt}
        ]
    )
    
    completion = response.choices[0].message.content
    
    # Generation aufzeichnen
    generation = trace.generation(
        name=f"variant_{variant}",
        model="gpt-3.5-turbo",
        prompt=prompt,
        completion=completion,
        usage={
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        }
    )
    
    # Trace abschließen
    trace.update(status="success")
    
    return {
        "trace_id": trace.id,
        "variant": variant,
        "completion": completion,
        "tokens": response.usage.total_tokens
    }

# Tests durchführen
result_a = run_test_variant("A", prompt_a)
result_b = run_test_variant("B", prompt_b)

# Ergebnisse anzeigen
print(f"Test-ID: {test_id}\n")

print(f"=== Variante A (Trace-ID: {result_a['trace_id']}) ===")
print(f"Tokens: {result_a['tokens']}")
print("Antwort:")
print(result_a['completion'][:200] + "..." if len(result_a['completion']) > 200 else result_a['completion'])

print(f"\n=== Variante B (Trace-ID: {result_b['trace_id']}) ===")
print(f"Tokens: {result_b['tokens']}")
print("Antwort:")
print(result_b['completion'][:200] + "..." if len(result_b['completion']) > 200 else result_b['completion'])

print("\nBeide Varianten sind in LangFuse verfügbar und können dort verglichen werden.")

## 11. Verwendung von LangFuse in einem Produktionskontext

In einem Produktionskontext sollten Sie die folgenden Best Practices beachten:

### Best Practices

1. **Strukturierte Tracing-Hierarchie**
   - Verwenden Sie konsistente Namenskonventionen für Traces, Spans und Generations
   - Ordnen Sie Spans logisch in einer hierarchischen Struktur an
   - Nutzen Sie Tags und Metadaten für eine einfache Filterung

2. **Fehlerbehandlung**
   - Zeichnen Sie Fehler explizit auf, indem Sie den Status auf `error` setzen
   - Fügen Sie Fehlermeldungen und Stack-Traces als Metadaten hinzu
   - Implementieren Sie Try-Catch-Blöcke um das Tracing

3. **Datenschutz**
   - Entfernen Sie personenbezogene Daten aus Prompts und Antworten
   - Verwenden Sie Filter oder Masking-Techniken für sensible Informationen
   - Setzen Sie `user_id` nur verschlüsselt oder anonymisiert ein

4. **Batch-Integration**
   - Für hohe Durchsatzraten: Verwenden Sie die Batch-API für eine effiziente Datenübermittlung
   - Implementieren Sie eine lokale Zwischenspeicherung für den Fall von Konnektivitätsproblemen

5. **Monitoring**
   - Richten Sie Alerts für außergewöhnliche Aktivitäten ein
   - Überwachen Sie die API-Nutzung und Kosten
   - Erstellen Sie Dashboards für wichtige Leistungsindikatoren

### Beispiel für eine Produktionsintegration

```python
from langfuse import Langfuse
import logging
from functools import wraps

# Konfiguration der Logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("llm_app")

# LangFuse-Client initialisieren
langfuse_client = Langfuse(
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY"),
    host=os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com")
)

# Decorator für Tracing von Funktionen
def trace_function(name=None, tags=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            func_name = name or func.__name__
            trace = langfuse_client.trace(name=func_name, tags=tags or [])
            
            try:
                with trace.span(name=f"{func_name}_execution") as span:
                    result = func(*args, **kwargs, trace=trace)
                    trace.update(status="success")
                    return result
            except Exception as e:
                logger.exception(f"Error in {func_name}: {str(e)}")
                trace.update(
                    status="error",
                    metadata={
                        "error": str(e),
                        "error_type": type(e).__name__
                    }
                )
                raise
        return wrapper
    return decorator

# Beispiel für eine Funktion mit Tracing
@trace_function(name="process_customer_query", tags=["production", "customer_service"])
def process_customer_query(query, customer_id, trace=None):
    # Anonymisieren von Daten
    safe_query = anonymize_pii(query)  # Fiktive Funktion zur PII-Entfernung
    
    # LLM-Verarbeitung mit Tracing
    with trace.span(name="query_processing") as processing_span:
        # LLM-Verarbeitung hier...
        response = "Antwort auf die Kundenanfrage"
        
    return response
```

## 12. Übungen

Versuchen Sie, die folgenden Übungen zu lösen, um Ihr Verständnis von LangFuse zu vertiefen.

### Übung 1: Grundlegende Trace-Erstellung

Erstellen Sie einen Trace, der eine einfache Konversation mit einem LLM verfolgt. Die Konversation sollte mindestens 3 Nachrichtenwechsel enthalten, und Sie sollten für jede Nachricht eine separate Generation aufzeichnen.

### Übung 2: Leistungsvergleich

Vergleichen Sie die Leistung von zwei verschiedenen LLM-Modellen (z.B. GPT-3.5-Turbo und GPT-4) für die gleiche Aufgabe. Zeichnen Sie Metriken wie Antwortzeit, Token-Verbrauch und Antwortqualität auf.

### Übung 3: Integration mit eigenen Workflows

Integrieren Sie LangFuse in einen eigenen LangChain-Workflow Ihrer Wahl. Der Workflow sollte mindestens einen Agenten oder eine Chain verwenden und mindestens ein Tool einbinden.

### Übung 4: Automatisierte Qualitätsbewertung

Implementieren Sie eine automatisierte Qualitätsbewertung für LLM-Antworten mit LangFuse. Verwenden Sie dazu das LLM-as-a-Judge-Konzept, um die Qualität von Antworten auf einer Skala von 1-10 zu bewerten.

## 13. Zusätzliche Ressourcen

- [LangFuse Dokumentation](https://langfuse.com/docs)
- [LangFuse GitHub Repository](https://github.com/langfuse/langfuse)
- [LangChain-LangFuse Integration](https://python.langchain.com/docs/integrations/callbacks/langfuse)
- [Blog: Using LangFuse to Debug and Optimize LLM Applications](https://langfuse.com/blog)
- [Discord Community](https://discord.gg/7NXusRtVa8)

## Zusammenfassung

In diesem Notebook haben Sie gelernt:

- Wie LangFuse für das Monitoring und Tracing von LLM-Anwendungen eingesetzt wird
- Wie Sie einfaches und fortgeschrittenes Tracing implementieren können
- Wie LangFuse mit LangChain integriert werden kann
- Wie Sie Metriken und Evaluierungen für Ihre LLM-Anwendungen erfassen können
- Wie Sie A/B-Tests mit LangFuse durchführen können
- Best Practices für den Einsatz von LangFuse in Produktionsumgebungen

LangFuse ist ein leistungsstarkes Werkzeug, das Ihnen hilft, Ihre LLM-Anwendungen besser zu verstehen, zu debuggen und zu optimieren. Durch die Integration von LangFuse in Ihren Entwicklungsworkflow können Sie die Qualität Ihrer KI-Anwendungen kontinuierlich verbessern und gleichzeitig Kosten und Leistung im Auge behalten.